In [1]:
from src.data import ForexData
from src.Technicals import Indicator
from src.Trademanager import Trademanager

In [2]:
df = ForexData().fetch_data(instrument="EUR_USD")

In [3]:
baseline_df = Indicator().Baseline(data=df, period=10)
ssl_df = Indicator().SSL(data=baseline_df, period=10)
atr_df = Indicator().ATR(data=ssl_df,timeperiod=14)
wae_df=Indicator().WAE(df=atr_df)

In [4]:
wae_df

,instrument,Time,granularity,Open,High,Low,Close,baseline,smaHigh,smaLow,...,TP,macd,std,bb_upper,bb_lower,t1,e1,trendUp,trendDown,FINAL_SIGNAL
0,EUR_USD,2023-09-29 21:00:00+10:00,H1,1.06094,1.06112,1.05902,1.05944,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,0.000000,None
1,EUR_USD,2023-09-29 22:00:00+10:00,H1,1.05945,1.06106,1.05883,1.05999,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,0.000000,None
2,EUR_USD,2023-09-29 23:00:00+10:00,H1,1.06000,1.06074,1.05746,1.05810,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,0.000000,None
3,EUR_USD,2023-09-30 00:00:00+10:00,H1,1.05808,1.05929,1.05700,1.05871,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,0.000000,None
4,EUR_USD,2023-09-30 01:00:00+10:00,H1,1.05869,1.05904,1.05728,1.05834,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,0.000000,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4995,EUR_USD,2024-07-20 02:00:00+10:00,H1,1.08880,1.08912,1.08840,1.08850,1.088582,1.088984,1.088088,...,1.089295,-0.000803,0.000566,1.089714,1.087450,0.001060,0.002264,0.001060,0.000000,None
4996,EUR_USD,2024-07-20 03:00:00+10:00,H1,1.08850,1.08855,1.08808,1.08839,1.088542,1.088933,1.088078,...,1.089162,-0.000798,0.000532,1.089606,1.087478,0.000705,0.002128,0.000705,0.000000,None
4997,EUR_USD,2024-07-20 04:00:00+10:00,H1,1.08838,1.08846,1.08758,1.08788,1.088538,1.088900,1.088074,...,1.087101,-0.000814,0.000521,1.089580,1.087496,-0.002391,0.002084,0.000000,0.002391,SELL
4998,EUR_USD,2024-07-20 05:00:00+10:00,H1,1.08790,1.08822,1.08768,1.08794,1.088528,1.088875,1.088067,...,1.087178,-0.000821,0.000446,1.089420,1.087636,-0.001073,0.001783,0.000000,0.001073,None


In [6]:
def backtesting_strategy(df):
    baseline_df = Indicator().Baseline(data=df, period=10)
    ssl_df = Indicator().SSL(data=baseline_df, period=10)
    atr_df = Indicator().ATR(data=ssl_df,timeperiod=14)
    wae_df=Indicator().WAE(df=atr_df)
    def final_signal(row):
        long_conditon = row['Close'] > row['baseline'] and row['SSL_BUY'] and row['trendUp']>row['e1']
        short_conditon = row['Close'] < row['baseline'] and row['SSL_SELL'] and row['trendDown']>row['e1']

        if long_conditon:
            return "BUY"
        elif short_conditon:
            return "SELL"
        else:
            return None
    wae_df['FINAL_SIGNAL'] = wae_df.apply(final_signal,axis = 1)
    return wae_df        

In [7]:
back_test_df = backtesting_strategy(df=df)

In [8]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import numpy as np  # Import numpy for np.where function

def create_figure_sig(sig):
    # Initialize figure with subplots
    fig_ssl = make_subplots(rows=2, cols=1, shared_xaxes=True, vertical_spacing=0.05)

    # Candlestick chart
    fig_ssl.add_trace(go.Candlestick(
        x=sig['Time'],
        open=sig['Open'],
        high=sig['High'],
        low=sig['Low'],
        close=sig['Close'],
        name='Candlestick',
        line=dict(width=1),
        opacity=1,
        increasing_fillcolor='#24A06B',
        decreasing_fillcolor="#CC2E3C",
        increasing_line_color='#2EC886',
        decreasing_line_color='#FF3A4C'
    ), row=1, col=1)

    #TP and SL adder
    fig_ssl.add_trace(go.Scatter(

    ))

    # SSLUp, SSLDown, Baseline traces
    fig_ssl.add_trace(go.Scatter(
        x=sig['Time'],
        y=sig['sslUp'],
        name='sslUp',
        mode='lines',
        line=dict(color='green', width=2)
    ), row=1, col=1)

    fig_ssl.add_trace(go.Scatter(
        x=sig['Time'],
        y=sig['sslDown'],
        name='sslDown',
        mode='lines',
        line=dict(color='red', width=2)
    ), row=1, col=1)

    fig_ssl.add_trace(go.Scatter(
        x=sig['Time'],
        y=sig['baseline'],
        name='Baseline',
        mode='lines',
        line=dict(color='yellow', width=2)
    ), row=1, col=1)

    # Buy and Sell signals
    fig_ssl.add_trace(go.Scatter(
        x=sig[sig['FINAL_SIGNAL'] == "BUY"]['Time'],
        y=sig[sig['FINAL_SIGNAL'] == "BUY"]['Open'],
        name='BUY_ARROW',
        mode='markers',
        marker=dict(color='yellow', symbol='triangle-up', size=10)
    ), row=1, col=1)

    fig_ssl.add_trace(go.Scatter(
        x=sig[sig['FINAL_SIGNAL'] == "SELL"]['Time'],
        y=sig[sig['FINAL_SIGNAL'] == "SELL"]['Open'],
        name='SELL_ARROW',
        mode='markers',
        marker=dict(color='red', symbol='triangle-down', size=10)
    ), row=1, col=1)

    # WAE chart
    fig_ssl.add_trace(go.Bar(
        x=sig['Time'],
        y=sig['trendUp'],
        marker_color=np.where(sig['trendUp'] < sig['trendUp'].shift(1), 'lime', 'green'),
        name='UpTrend',
        width=0.04,
        marker_line_color='rgba(0,0,0,0)',
        marker_line_width=0,
    ), row=2, col=1)

    fig_ssl.add_trace(go.Bar(
        x=sig['Time'],
        y=sig['trendDown'],
        marker_color=np.where(sig['trendDown'] < sig['trendDown'].shift(1), 'orange', 'red'),
        name='DownTrend',
        width=0.04,
        marker_line_color='rgba(0,0,0,0)',
        marker_line_width=0,
    ), row=2, col=1)

    fig_ssl.add_trace(go.Scatter(
        x=sig['Time'],
        y=sig['e1'],
        mode='lines',
        name='ExplosionLine',
        line=dict(color='#A0522D', width=2)
    ), row=2, col=1)

    # Update layout for the entire figure
    fig_ssl.update_layout(
        title='Candlestick Chart with WAE',
        xaxis_title='Time',
        yaxis_title='Price',
        height=800,
        width=1200,
        margin=dict(l=50, r=50, b=50, t=100),
        paper_bgcolor="#1e1e1e",
        plot_bgcolor="#1e1e1e",
        font=dict(size=10, color="#e1e1e1"),
        showlegend=True,
        legend=dict(x=0, y=1.1, orientation='h')
    )

    # Show gridlines
    fig_ssl.update_xaxes(
        gridcolor="#1f292f",
        showgrid=True,
        fixedrange=True,
        rangeslider=dict(visible=True)
    )

    fig_ssl.update_yaxes(
        gridcolor="#1f292f",
        showgrid=True
    )

    return fig_ssl

# Example usage:
# Assuming sig is your DataFrame with necessary columns ('Time', 'Open', 'High', 'Low', 'Close', 'sslUp', 'sslDown', 'baseline', 'FINAL_SIG', 'trendUp', 'trendDown', 'e1')
# Create the figure using the function
fig = create_figure_sig(back_test_df)

# Show the figure
fig.show()


In [9]:
checkng = back_test_df.iloc[500:1000]

In [10]:
checkng

,instrument,Time,granularity,Open,High,Low,Close,baseline,smaHigh,smaLow,...,TP,macd,std,bb_upper,bb_lower,t1,e1,trendUp,trendDown,FINAL_SIGNAL
500,EUR_USD,2023-10-30 18:00:00+11:00,H1,1.05650,1.05676,1.05530,1.05568,1.056176,1.056475,1.055804,...,1.054618,-0.000055,0.000919,1.058014,1.054338,-0.004886,0.003676,0.000000,0.004886,SELL
501,EUR_USD,2023-10-30 19:00:00+11:00,H1,1.05555,1.05608,1.05472,1.05600,1.056163,1.056397,1.055686,...,1.057084,-0.000068,0.000897,1.057957,1.054369,-0.001937,0.003588,0.000000,0.001937,None
502,EUR_USD,2023-10-30 20:00:00+11:00,H1,1.05599,1.05862,1.05599,1.05826,1.056369,1.056619,1.055695,...,1.059454,0.000026,0.000917,1.058203,1.054535,0.014129,0.003668,0.014129,0.000000,None
503,EUR_USD,2023-10-30 21:00:00+11:00,H1,1.05824,1.05882,1.05740,1.05749,1.056492,1.056870,1.055882,...,1.058700,0.000072,0.000898,1.058289,1.054695,0.006776,0.003593,0.006776,0.000000,None
504,EUR_USD,2023-10-30 22:00:00+11:00,H1,1.05750,1.05965,1.05734,1.05855,1.056708,1.057195,1.056021,...,1.059839,0.000159,0.000825,1.058357,1.055059,0.013168,0.003299,0.013168,0.000000,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,EUR_USD,2023-11-28 10:00:00+11:00,H1,1.09544,1.09594,1.09544,1.09592,1.094553,1.094913,1.093887,...,1.096923,0.000731,0.000873,1.096298,1.092808,0.004623,0.003491,0.004623,0.000000,None
996,EUR_USD,2023-11-28 11:00:00+11:00,H1,1.09590,1.09618,1.09553,1.09582,1.094851,1.095080,1.094186,...,1.096798,0.000750,0.000907,1.096665,1.093037,0.002772,0.003627,0.002772,0.000000,None
997,EUR_USD,2023-11-28 12:00:00+11:00,H1,1.09582,1.09620,1.09542,1.09578,1.095103,1.095306,1.094478,...,1.096744,0.000761,0.000934,1.096971,1.093235,0.001584,0.003736,0.001584,0.000000,None
998,EUR_USD,2023-11-28 13:00:00+11:00,H1,1.09577,1.09630,1.09534,1.09578,1.095316,1.095556,1.094734,...,1.096744,0.000766,0.000958,1.097232,1.093400,0.000833,0.003832,0.000833,0.000000,None


In [11]:
create_figure_sig(checkng)

In [13]:
def backtest_func(df):
  entry_price = None
  trade_type = None
  pips_gained = []
  for index, row in df.iterrows():
    if row['FINAL_SIGNAL'] == 'SELL' or row['FINAL_SIGNAL'] == 'BUY':
      if entry_price is None:
        entry_price = row['Close']
        trade_type = row['FINAL_SIGNAL']
        print(entry_price,trade_type)
      elif entry_price is not None:

        exit_price = row['Close']
        pips = exit_price - entry_price if trade_type == "SELL" else entry_price - exit_price
        pips_gained.append(abs(pips))
        print(exit_price,entry_price,abs(pips))

        entry_price = row['Close']
        trade_type = row['FINAL_SIGNAL']
        print(entry_price,trade_type)

  return sum(pips_gained)






In [14]:

bd = backtest_func(wae_df)

1.04812 BUY
1.04702 1.04812 0.0010999999999998789
1.04702 BUY
1.04848 1.04702 0.0014600000000000168
1.04848 BUY
1.05072 1.04848 0.0022400000000000198
1.05072 SELL
1.05188 1.05072 0.0011599999999998278
1.05188 BUY
1.04963 1.05188 0.0022499999999998632
1.04963 SELL
1.06016 1.04963 0.010529999999999928
1.06016 SELL
1.05999 1.06016 0.00017000000000000348
1.05999 SELL
1.05932 1.05999 0.0006699999999999484
1.05932 SELL
1.06202 1.05932 0.0026999999999999247
1.06202 SELL
1.06182 1.06202 0.00019999999999997797
1.06182 SELL
1.05294 1.06182 0.008879999999999999
1.05294 SELL
1.0536 1.05294 0.000660000000000105
1.0536 SELL
1.0554 1.0536 0.0017999999999998018
1.0554 BUY
1.05644 1.0554 0.001040000000000152
1.05644 SELL
1.05692 1.05644 0.00048000000000003595
1.05692 SELL
1.0536 1.05692 0.0033199999999999896
1.0536 SELL
1.05706 1.0536 0.0034600000000000186
1.05706 SELL
1.05877 1.05706 0.0017099999999998783
1.05877 BUY
1.05872 1.05877 4.999999999988347e-05
1.05872 SELL
1.06631 1.05872 0.0075899999999999

In [15]:
print(bd)

0.6084799999999968
